In [1]:
import numpy as np
from scipy import optimize as opt

In [9]:
# Model Primitives
np.random.seed(1234567890)

nObs = 1000
beta = np.array([0.5, 0.5] , dtype=float)
income= np.random.uniform(size = nObs) # draws from standard normal
explVar = np.vstack([np.ones(nObs), income]).T

In [41]:
def simulateBinaryLogit(x, beta):
    nObs     = x.shape[0]
    nChoice  = 2;
    
    epsilon = np.random.gumbel(size = [nObs, nChoice])
    beta_augmented = np.vstack([np.zeros(beta.shape), beta])
    utility = x @ beta_augmented.T + epsilon
    return np.argmax(utility, axis=1)

In [42]:
choice = simulateBinaryLogit(explVar, beta)

choice[1:10]

array([0, 1, 1, 0, 1, 0, 1, 1, 1], dtype=int64)

In [12]:
data = np.hstack((choice.reshape(nObs, 1), explVar))
data.shape

(1000, 3)

In [13]:
def calcLambda(x, beta):
    prob = np.exp(x @ beta)  / (1 + np.exp(x @ beta))
    return prob

In [110]:
def logLike_binaryLogit(beta, y, x):
    choiceProb   = calcLambda(x, beta);
    
    ll_i         = y * np.log(choiceProb) + (1 - y) * np.log(1 - choiceProb);
    logLike      = -(ll_i.sum())
    return logLike

In [161]:
beta0 = np.zeros(2)
out = opt.minimize(logLike_binaryLogit, beta0, args=(data[:,0], data[:,1:]) , method='L-BFGS-B', tol=1e-12)

print('beta hat is:', out.x)

print('value of likelihood at beta:', out.fun)

# how to get SE's?
out

beta hat is: [ 0.40157312  0.5869323 ]
value of likelihood at beta: 633.200201176


      fun: 633.20020117642594
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 30
      nit: 8
   status: 0
  success: True
        x: array([ 0.40157312,  0.5869323 ])

In [155]:
# Simulated Maximum Likelihood

def logLikeSim_binaryLogit(beta, y, x, nSim):
    np.random.seed(42)
    
    nObs = y.shape[0]
    
    simChoice = np.empty((nObs,nSim))
    simChoice[:] = np.NAN
    
    for iSim in range(0, nSim):
        simChoice[:,iSim] = simulateBinaryLogit(x, beta)
    
    simProb = simChoice.mean(axis=1)
    
    ll_i         = y * np.log(simProb) + (1 - y) * np.log(1 - simProb)
    logLike      = -(ll_i.sum())
    return logLike
    


In [186]:
beta0 = 0.4*np.zeros(2)
nSim = 1000

out = opt.minimize(logLikeSim_binaryLogit, beta0, args=(data[:,0], data[:,1:], nSim) , method='SLSQP', \
             options={'gtol': 1e-4, 'eps': 1e-04, 'ftol': 1e-8})

print('beta hat is:', out.x)

print('value of likelihood at beta:', out.fun)

# how to get SE's?
out

C:\Users\ldeer\Anaconda\lib\site-packages\ipykernel\__main__.py:4: OptimizeWarning: Unknown solver options: gtol
C:\Users\ldeer\Anaconda\lib\site-packages\ipykernel\__main__.py:16: RuntimeWarning: divide by zero encountered in log
C:\Users\ldeer\Anaconda\lib\site-packages\ipykernel\__main__.py:16: RuntimeWarning: invalid value encountered in multiply


beta hat is: [ 0.48769872  0.44145953]
value of likelihood at beta: 634.2116481


     fun: 634.21164810015466
     jac: array([ 153.93409617,   -3.49224709,    0.        ])
 message: 'Optimization terminated successfully.'
    nfev: 61
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([ 0.48769872,  0.44145953])

In [187]:
out.jac

array([ 153.93409617,   -3.49224709,    0.        ])

In [188]:
## Compare to the canned logit model 

import statsmodels.api as sm

logit_mod = sm.Logit(data[:,0], data[:,1:])
logit_res = logit_mod.fit(disp=0)
print('Parameters: ', logit_res.params)

Parameters:  [ 0.40157314  0.58693222]


In [189]:
logit_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      998
Method:                           MLE   Df Model:                            1
Date:                Fri, 14 Apr 2017   Pseudo R-squ.:                0.004845
Time:                        07:50:16   Log-Likelihood:                -633.20
converged:                       True   LL-Null:                       -636.28
                                        LLR p-value:                   0.01303
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.4016      0.135      2.985      0.003         0.138     0.665
x1             0.5869      0.237      2.477      0.013         0.122     1.051
==============================================================================
"""

In [ ]:
## use the Generic Likelihood class?


In [190]:
from statsmodels.base.model import GenericLikelihoodModel


In [196]:
class MyLogit(GenericLikelihoodModel):
    def loglike(self, params):
        exog = self.exog
        endog = self.endog
        
        choiceProb   = calcLambda(self.exog, params);
        ll_i         = endog * np.log(choiceProb) + (1 - endog) * np.log(1 - choiceProb);
        return ll_i.sum()

In [197]:
sm_logit_manual = MyLogit(data[:,0], data[:,1:]).fit()
print(sm_logit_manual.summary())

Optimization terminated successfully.
         Current function value: 0.633200
         Iterations: 57
         Function evaluations: 112
                               MyLogit Results                                
Dep. Variable:                      y   Log-Likelihood:                -633.20
Model:                        MyLogit   AIC:                             1270.
Method:            Maximum Likelihood   BIC:                             1280.
Date:                Fri, 14 Apr 2017                                         
Time:                        07:58:40                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------

In [ ]:
# Simulated ML in the Generic Class?